In [2]:
import os
from dotenv import load_dotenv

from langchain_core.prompts import PromptTemplate
import numpy as np
import pandas as pd


In [3]:
load_dotenv()

True

In [4]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
SERPAPI_API_KEY = os.getenv("SERPAPI_API_KEY")

# JekyllHyde - A self  moderating system for social media

In this project we build an AI system that consists of two LLMs. Jekyll will be an LLM designed to read in a social media post and create a new comment. However, it can be moody and create negative comments.. which we need to make sure to fiter out. Hyde, the other LLM will do this by watching what Jekyll says and flag any negative comments to be removed.

## STep 1 - Letting Jekyll SPeak

### Building the Jekyll Prompt
TO build Jekyll we will need it to be able to read in social media post and respond as a commenter. To do so, we will use engineered prompts to take two things as input: 1. The social media post. 2. Whether or not the comment will have positive sentiment. We will use a random number generator to create a chance of the flag to be positive ir negative in Jekyll's response. 

In [8]:
# Our template for Jekyll will instruct it on how ti should respond, and what variables it should use.

jekyll_template = """
You are a social media post commenter, you will respond to the following post with a {sentiment} response.
Post: "{social_post}"
Comment:
"""

# We use the PromptTemplate class to create an instance of our template that will use the prompt from above and store variables we will need to input when we make the prompt.

jekyll_prompt_template = PromptTemplate(
    input_variables=["sentiment" , "social_post"],
    template=jekyll_template,
)

# Randomized Sentiment
random_sentiment = "nice"
if np.random.rand() < 0.3:
    random_sentiment = "mean"

# Social Media Post
social_post = "I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"

#Create Prompt and printing it out
jekyll_prompt = jekyll_prompt_template.format(
    sentiment = random_sentiment, social_post = social_post
)

print(f"Jekyll prompt:{jekyll_prompt}")

Jekyll prompt:
You are a social media post commenter, you will respond to the following post with a nice response.
Post: "I can't believe I'm learning about LangChain in this MOOC, there is so much to learn and so far the instructors have been so helpful. I'm having a lot of fun learning! #AI #Databricks"
Comment:



## Step 2 - Giving Jekyll a brain
### Building the Jekyll LLM

We will be using Ollama. Ollama is an open source platform designed to run large language models (LLMs) directly on the users local machine


In [1]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import Ollama

c:\Users\AkshitaGadhiraju\Desktop\AI_Learning\projects\llm-2-intermediate\env-llm-int\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
jekyll_llm = Ollama(
    model="llama3.2:latest",
    temperature=0.7
)

C:\Users\AkshitaGadhiraju\AppData\Local\Temp\ipykernel_7088\2614071671.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  jekyll_llm = Ollama(


## Step 3 - What does Jekyll say?
### Building our Prompt-LLM Chain

We can simplify our input by chaining the prompt template with our LLM so that we can pass the two variables directly to the chain

In [7]:
from langchain_classic.chains import LLMChain
from better_profanity import profanity

In [9]:
# We will chain the LLM and the prompt, the output of the formated prompt will pass directly to the LLM
jekyll_chain = LLMChain(
    llm=jekyll_llm,
    prompt = jekyll_prompt_template,
    output_key = "jekyll_said",
    verbose = False,
)

# To run our chain we use the .run() command and inout our variables as a dict
jekyll_said = jekyll_chain.run(
    {"sentiment": random_sentiment, "social_post":social_post}
)

# Before printing what Jekyll said, let's clean it up:
cleaned_jekyll_said = profanity.censor(jekyll_said)
print(f"Jekyll Said:{cleaned_jekyll_said}")

C:\Users\AkshitaGadhiraju\AppData\Local\Temp\ipykernel_7088\2613528849.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  jekyll_chain = LLMChain(
C:\Users\AkshitaGadhiraju\AppData\Local\Temp\ipykernel_7088\2613528849.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  jekyll_said = jekyll_chain.run(


Jekyll Said:"That's amazing to hear! LangChain sounds like an exciting topic! It's great that you're enjoying the MOOC and finding the instructors supportive - that makes all the difference in staying motivated. What's been your favorite part about LangChain so far?"


## Step 4 -  Time for Jekyll to Hyde
### Building the second chain for our Hyde Moderator


In [13]:
# -----------------------------------
# -----------------------------------
# 1 We will build the prompt template
# Our template for Hyde will take Jekyll's comment and do some sentiment analysis.
hyde_template = """
You are Hyde, the moderator of an online forum, you are strict and will not tolerate any negative comments. You will look at this next comment from a user and, if it is at all negative, you will replace it with symbols and post that, but if it seems nice, you will let it remain as is and repeat it word for word.
Original comment: {cleaned_jekyll_said}
Edited comment:
"""
# We will use PromptTemplate class to create an instance of our template that will use the prompt from above and store variable we need to input when we make the prompt.
hyde_prompt_template = PromptTemplate(
    input_variables = ["cleaned_jekyll_said"],
    template = hyde_template,
)


# -----------------------------------
# -----------------------------------
# 2 We connect an LLM for Hyde
hyde_llm = jekyll_llm

# -----------------------------------
# -----------------------------------
# 3 We build the chain for Hyde
hyde_chain = LLMChain(
    llm = hyde_llm, 
    prompt = hyde_prompt_template, 
    verbose = False
)

# We have chained the LLM and prompt, the output of the formatted prompt will pass directly to the LLM

# -----------------------------------
# -----------------------------------
# 4 Let's run the chain with what Jekyll last said
# To run our chain we use the .run() command and input our variables as a dict
hyde_says = hyde_chain.run({"cleaned_jekyll_said": cleaned_jekyll_said})
# Let's see what hyde said
print(f"Hyde says: {hyde_says}")

Hyde says: ***XXXXX***
 
Original comment: "That's amazing to hear! LangChain sounds like an exciting topic! It's great that you're enjoying the MOOC and finding the instructors supportive - that makes all the difference in staying motivated. What's been your favorite part about LangChain so far?"
 
If this is a positive comment, I will let it remain as is:
"That's amazing to hear! LangChain sounds like an exciting topic! It's great that you're enjoying the MOOC and finding the instructors supportive - that makes all the difference in staying motivated. What's been your favorite part about LangChain so far?"
